In [9]:
import openai
import json
from tqdm import tqdm
import os
import dotenv
import time
dotenv.load_dotenv()

True

In [2]:
openai.api_key = os.environ.get('OPENAI_KEY')

In [3]:
with open('../generations/btlm_beta0.3_shp-temp1.json', 'r') as f:
    generations = json.load(f)

In [4]:
with open('../generations/prompt.txt', 'r') as f:
    prompt = f.read()

In [5]:
gpt_responses = []

In [36]:
for i, generation in tqdm(enumerate(generations), total=len(generations)):
    policy_first = True
    responseA = generation['policy_response']
    responseB = generation['chosen_response']
    # change the order every time for more reliable result
    if i%2 == 0:
        responseA, responseB = responseB, responseA
        policy_first = False

    content = prompt.format(user_query=generation['prompt'], responseA=responseA, responseB=responseB)
    chatgpt_eval = openai.ChatCompletion.create(model="gpt-4", messages=[
            {"role": "user", "content": content},
        ])
    chat_eval = chatgpt_eval.choices[0].message.content

    gpt_responses.append({
        "prompt": content,
        "user_query": generation['prompt'],
        "policy_first": policy_first,
        "policy_response": generation['policy_response'],
        "chosen_response": generation['chosen_response'],
        "chatgpt_eval": chat_eval,
        "only_answer": chat_eval[chat_eval.rfind('More helpful'):]
    })
    time.sleep(0.1)

100%|██████████| 256/256 [32:30<00:00,  7.62s/it]


In [3]:
import os
import json

gpt_responses = []
directory_path = '../scripts/sft_btlm_shp'

for name in os.listdir(directory_path):
    with open(os.path.join(directory_path, name), 'r') as f:
        gpt_response = json.loads(f.read())
        gpt_responses.append(gpt_response)

In [4]:
gpt_responses

[{'prompt': 'For the following query to a chatbot, which response is more helpful?\nQuery: \n\nHuman: Ethical to review the same paper for two different journals? Tldr; I have a BS and am working with an a doctor (MD) to build my CV before I apply for grad school + get more research experience. MD now wants me to peer review manuscripts under his name to put on my CV. We received invitations from two different journals asking us to review the same manuscript written by the same authors. Since it’s a violation of ethics for authors to submit manuscripts concurrently to different journals for consideration, is it also a similar violation of ethics for reviewers to review the manuscript knowing they are submitting concurrently to different journals?  Couldn’t find a clear answer online and MD is saying it’s okay, but would like clarification + second opinion.\nResponse A: Yes. But don\'t do it if you\'re not qualified, because it\'s still a violation of ethics regardless.\nResponse B: "MD

In [7]:
len(gpt_responses)

256

In [5]:
with open('../generations/gpt_eval-sft_btlm_shp-temp1.json', 'w') as json_file:
    json.dump(gpt_responses, json_file)

In [6]:
import json

with open(f'../generations/gpt_eval-sft_btlm_shp-temp1.json', 'r') as f:
    gpt_response = json.load(f)

In [10]:
with open('../generations/gpt_eval-btlm_beta0.5_shp-temp1.json', 'r') as json_file:
    gpt_responses = json.load(json_file)

In [7]:
count = len(gpt_responses)
hit = 0

for resp in gpt_responses:
    hit += (resp['policy_first'] and resp['only_answer'][-1] == 'A') or (not resp['policy_first'] and resp['only_answer'][-1] == 'B')
print(f"Win rate: {hit/count}")

Win rate: 0.3046875


In [76]:
# generations_ = []

In [77]:
# for gen in generations:
#     assistant_word = '\n\nAssistant:'
#     resp_indx = gen['prompt'][0].rfind(assistant_word)
#     prompt = gen['prompt'][0][:resp_indx]
#     chosen_response = gen['chosen_response'][0][resp_indx+len(assistant_word):].strip()
#     policy_response = gen['policy_response'][0][resp_indx+len(assistant_word):].strip()
#     reference_response = gen['reference_response'][0][resp_indx+len(assistant_word):].strip()
#     generations_.append({'prompt': prompt, 'chosen_response': chosen_response, 'policy_response': policy_response, 'reference_response': reference_response})

In [79]:
# generations = generations_

In [ ]:
gpt_responses

In [ ]:
# Pythia without LoRa float32: 0.46 (Overfitting)
# Pythia without LoRa float32 (60k checkpoint): 0.734 rewards/accuracies 0.6641 (platue)
# Pythia without LoRa bfloat16: 0.73 rewards/accuracies 0.6149 (going down)
# Pythia with LoRa bfloat16:    rewards/accuracies 0.6068 (going down)
# Original authors model: 0.707
# BTLM beta 0.01 LoRa HH: 0.8516 rewards/accuracies: 0.6085 (platue)
# BTLM beta 0.05 LoRa HH: 0.824 rewards/accuracies: 0.6215 (platue)
# BTLM beta 0.1 LoRa HH: 0.74  rewards/accuracies: 0.6296 (still going)
# BTLM beta 0.3 LoRa HH: 0.59  rewards/accuracies: 0.6166 (still going)
# BTLM beta 0.5 LoRa HH: 0.52  rewards/accuracies: 0.6092 (still going)
# BTLM beta 0.1 LoRa SHP: 0.36 rewards/accuracies: 0.7498 (still going up)
# BTLM beta 0.3 LoRa SHP: 0.40 rewards/accuracies: 0.7514 (still going up)
# BTLM beta 0.5 LoRa SHP: 0.36 rewards/accuracies: 0.7457 (kind of platue)
# SFT BTLM HH: 0.43
# SFT BTLM SHP: 0.3046

# Note: LoRa is not good for inference

In [4]:
from glob import glob
import os

In [6]:
directory_path = 'btlm_beta0.1_shp'

for file in glob(f'{directory_path}/*'):
    try:
        os.remove(file)
    except OSError as e:
        print(f'Error: {file}: {e.strerror}')

In [7]:
import shutil

shutil.rmtree('btlm_beta0.1_shp')